In [1]:
cd ../py_scripts/

/Users/r0b00sj/Documents/Capstone/Capstone/py_scripts


In [64]:
import pandas as pd
from gensim.models.coherencemodel import CoherenceModel
from utils_streamers import DirFileMgr, CorpStreamer, BOWCorpStreamer
from gensim import corpora
from gensim.models.ldamodel import LdaModel
import json
import pyLDAvis
import pyLDAvis.gensim

In [49]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
rootLogger = logging.getLogger()
rootLogger.setLevel(logging.INFO)

pd.options.display.max_colwidth = 500

In [4]:
dr_id_str = "dr_500k"
dr_fps = DirFileMgr(dr_id_str)
dr_fps.create_all_dr_fps(new_setup='N')

Which data set will be used? Enter either '5000' or 'full': full
corpus lst fp is assigned as  ../../outputs-git_ignored/dr_500k/dr_500k_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/dr_500k/dr_500k.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/dr_500k/dr_500k_json.txt
dimensional reduction run parameters fp is assigned as  ../../outputs-git_ignored/dr_500k/dr_500k_dr_run_params.txt


In [6]:
d = corpora.Dictionary.load(dr_fps.dictionary_fp)
c = CorpStreamer(d, dr_fps.corp_lst_fp, inc_title='Y')
corp_lst_stream = CorpStreamer(dictionary=d, corp_lst_fp=dr_fps.corp_lst_fp)

# bow_c = BOWCorpStreamer(d, dr_fps.corp_lst_fp, inc_title='Y')
# dr_fps.add_fp('gensim_corpus')

#long running time - execute only for a new dimensional reduction run
# if 0 == 1:
#     corpora.MmCorpus.serialize(dr_fps.gensim_corp_fp, bow_c)
    
# #whether the serialized corpus has just been already been created and saved (above), 
# # or was created and saved earlier (as for demo) we load it with this expression:
# serial_corp = corpora.MmCorpus(dr_fps.gensim_corp_fp)



2018-03-27 12:13:35,707 : INFO : loading Dictionary object from ../../outputs-git_ignored/dr_500k/dr_500k.dict
2018-03-27 12:13:35,884 : INFO : loaded ../../outputs-git_ignored/dr_500k/dr_500k.dict
2018-03-27 12:13:35,892 : INFO : loaded corpus index from ../../outputs-git_ignored/dr_500k/dr_500k_corpus.mm.index
2018-03-27 12:13:35,894 : INFO : initializing cython corpus reader from ../../outputs-git_ignored/dr_500k/dr_500k_corpus.mm
2018-03-27 12:13:35,899 : INFO : accepted corpus with 28492 documents, 262826 features, 90336941 non-zero entries


corpus fp is assigned as  ../../outputs-git_ignored/dr_500k/dr_500k_corpus.mm


In [65]:
json

<module 'json' from '/anaconda3/lib/python3.6/json/__init__.py'>

In [61]:
def analyze_model_coherence(mod_id_str, corp_lst_stream):   
    """
    :param {str} mod_id_str:
        The identifying string for the model to be analyzed
    :param {CorpStreamer} corp_lst_str:
        Streaming corpus object
    
    Loads and creates all objects to build a coherence model and associate it with the model topic terms.
    """
    #set up filepaths for this model for this task
    fps = DirFileMgr(mod_id_str)
    fps.create_all_modeling_fps(mod_id_str)
    fps.add_fp('pyLDAvis')
    fps.add_fp('coherence_model')
    fps.add_fp('mod_run_params')

    #load the trained LDA model
    lda = LdaModel.load(fps.model_fp)
    
    #load model run parameters
    with open(fps.mod_run_params) as run_params_f:
        run_params = json.load(run_params_f)
    
    #create and save pyLDAvis for visual inspection
    lda_vis_serialized = pyLDAvis.gensim.prepare(lda, serial_corp, d)
    pyLDAvis.save_html(lda_vis_serialized, fps.pyldavis_fp)

    #train and save coherence model
    cm_cv = CoherenceModel(model=lda, \
                        texts=corp_lst_stream, \
                        coherence='c_v')
    cm_cv.save(fps.coherence_model_fp + 'cv')

    #get coherence scores - for model overall, and for individual topics
    cv_coherence = cm_cv.get_coherence()
    cv_topic_coherence = cm_cv.get_coherence_per_topic()

    #create a dataframe that holds the relevant info for analyzing the model
    df = pd.DataFrame(data=lda.show_topics(num_topics=lda.num_topics), columns=['topic_num', 'topic_terms'])
    df.set_index(keys='topic_num', inplace=True)
    df['cv_coherence'] = cv_topic_coherence
    df.sort_values(by=['cv_coherence'], ascending=False, inplace=True)

    return df, cv_coherence, cv_topic_coherence, run_params

In [63]:
mod_1_df, mod_1_cv_coherence, mod_1_cv_topic_coherence, mod_1_run_params =\
    analyze_model_coherence('gpu_mod_run_1', corp_lst_stream)
    
print(mod_1_cv_coherence, mod_1_run_params)
mod_1_df

2018-03-28 12:21:07,000 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1.model
2018-03-28 12:21:07,020 : INFO : loading expElogbeta from ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1.model.expElogbeta.npy with mmap=None
2018-03-28 12:21:07,120 : INFO : setting ignored attribute dispatcher to None
2018-03-28 12:21:07,121 : INFO : setting ignored attribute state to None
2018-03-28 12:21:07,123 : INFO : setting ignored attribute id2word to None
2018-03-28 12:21:07,124 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1.model
2018-03-28 12:21:07,126 : INFO : dtype was not set in saved LdaMulticore file ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1.model, assuming np.float64
2018-03-28 12:21:07,128 : INFO : loading LdaState object from ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1.model.state
2018-03-28 12:21:07,136 : INFO : loading sstats from ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1.model.sta

corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1_mod_run_params.txt
pyLDAvis html fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1_ldavis.html
coherence model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1_coherence_model.
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1_mod_run_params.txt


2018-03-28 12:21:07,231 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1.model.state
2018-03-28 12:21:07,232 : INFO : dtype was not set in saved LdaState file ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1.model.state, assuming np.float64
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
2018-03-28 12:27:55,451 : INFO : saving CoherenceModel object under ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1_coherence_model.cv, separately None
2018-03-28 12:27:55,453 : INFO : storing np array 'sstats' to ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1_coherence_model.cv._model.state.sstats.npy
2018-03-28 12:27:55,522 : INFO : storing np array 'expElogbeta' to ../.

2018-03-28 12:30:35,987 : INFO : 62 batches submitted to accumulate stats from 3968 documents (35703571 virtual)
2018-03-28 12:30:43,768 : INFO : 63 batches submitted to accumulate stats from 4032 documents (36210981 virtual)
2018-03-28 12:30:44,393 : INFO : 64 batches submitted to accumulate stats from 4096 documents (36663722 virtual)
2018-03-28 12:30:45,207 : INFO : 65 batches submitted to accumulate stats from 4160 documents (37202399 virtual)
2018-03-28 12:30:46,225 : INFO : 66 batches submitted to accumulate stats from 4224 documents (37560385 virtual)
2018-03-28 12:30:48,431 : INFO : 67 batches submitted to accumulate stats from 4288 documents (38196793 virtual)
2018-03-28 12:30:50,972 : INFO : 68 batches submitted to accumulate stats from 4352 documents (38650823 virtual)
2018-03-28 12:30:54,827 : INFO : 69 batches submitted to accumulate stats from 4416 documents (39098416 virtual)
2018-03-28 12:30:59,162 : INFO : 70 batches submitted to accumulate stats from 4480 documents (3

2018-03-28 12:33:44,582 : INFO : 135 batches submitted to accumulate stats from 8640 documents (76260036 virtual)
2018-03-28 12:33:45,691 : INFO : 136 batches submitted to accumulate stats from 8704 documents (76750590 virtual)
2018-03-28 12:33:47,757 : INFO : 137 batches submitted to accumulate stats from 8768 documents (77411895 virtual)
2018-03-28 12:33:52,885 : INFO : 138 batches submitted to accumulate stats from 8832 documents (77996318 virtual)
2018-03-28 12:33:57,071 : INFO : 139 batches submitted to accumulate stats from 8896 documents (78525533 virtual)
2018-03-28 12:33:59,970 : INFO : 140 batches submitted to accumulate stats from 8960 documents (79115828 virtual)
2018-03-28 12:34:00,812 : INFO : 141 batches submitted to accumulate stats from 9024 documents (79690300 virtual)
2018-03-28 12:34:02,919 : INFO : 142 batches submitted to accumulate stats from 9088 documents (80250005 virtual)
2018-03-28 12:34:03,543 : INFO : 143 batches submitted to accumulate stats from 9152 doc

...

2018-03-28 12:39:26,935 : INFO : 278 batches submitted to accumulate stats from 17792 documents (146784834 virtual)
2018-03-28 12:39:29,007 : INFO : 279 batches submitted to accumulate stats from 17856 documents (147232939 virtual)
2018-03-28 12:39:30,409 : INFO : 280 batches submitted to accumulate stats from 17920 documents (147659090 virtual)
2018-03-28 12:39:31,321 : INFO : 281 batches submitted to accumulate stats from 17984 documents (148178356 virtual)
2018-03-28 12:39:33,539 : INFO : 282 batches submitted to accumulate stats from 18048 documents (148711007 virtual)
2018-03-28 12:39:34,057 : INFO : 283 batches submitted to accumulate stats from 18112 documents (149103597 virtual)
2018-03-28 12:39:35,612 : INFO : 284 batches submitted to accumulate stats from 18176 documents (149423813 virtual)
2018-03-28 12:39:42,279 : INFO : 285 batches submitted to accumulate stats from 18240 documents (149836253 virtual)
2018-03-28 12:39:43,800 : INFO : 286 batches submitted to accumulate sta

2018-03-28 12:42:18,087 : INFO : 349 batches submitted to accumulate stats from 22336 documents (185364673 virtual)
2018-03-28 12:42:21,868 : INFO : 350 batches submitted to accumulate stats from 22400 documents (185891121 virtual)
2018-03-28 12:42:25,102 : INFO : 351 batches submitted to accumulate stats from 22464 documents (186313818 virtual)
2018-03-28 12:42:25,375 : INFO : 352 batches submitted to accumulate stats from 22528 documents (186767594 virtual)
2018-03-28 12:42:27,708 : INFO : 353 batches submitted to accumulate stats from 22592 documents (187256362 virtual)
2018-03-28 12:42:31,999 : INFO : 354 batches submitted to accumulate stats from 22656 documents (187708641 virtual)
2018-03-28 12:42:38,034 : INFO : 355 batches submitted to accumulate stats from 22720 documents (188310645 virtual)
2018-03-28 12:42:39,372 : INFO : 356 batches submitted to accumulate stats from 22784 documents (188856412 virtual)
2018-03-28 12:42:44,758 : INFO : 357 batches submitted to accumulate sta

2018-03-28 12:45:39,243 : INFO : 420 batches submitted to accumulate stats from 26880 documents (228405381 virtual)
2018-03-28 12:45:42,654 : INFO : 421 batches submitted to accumulate stats from 26944 documents (228875888 virtual)
2018-03-28 12:45:43,303 : INFO : 422 batches submitted to accumulate stats from 27008 documents (229520266 virtual)
2018-03-28 12:45:43,801 : INFO : 423 batches submitted to accumulate stats from 27072 documents (230260048 virtual)
2018-03-28 12:45:52,081 : INFO : 424 batches submitted to accumulate stats from 27136 documents (230868972 virtual)
2018-03-28 12:45:55,944 : INFO : 425 batches submitted to accumulate stats from 27200 documents (231490388 virtual)
2018-03-28 12:45:56,424 : INFO : 426 batches submitted to accumulate stats from 27264 documents (232133623 virtual)
2018-03-28 12:45:59,172 : INFO : 427 batches submitted to accumulate stats from 27328 documents (232778899 virtual)
2018-03-28 12:46:01,851 : INFO : 428 batches submitted to accumulate sta

0.4412697789628097 {'num_topics': 50, 'cores': 2, 'chunksize': 2000, 'passes': 10, 'batch': False, 'alpha': 'symmetric', 'eta': None, 'decay': 0.5, 'offset': 1.0, 'eval_every': 10, 'iterations': 50, 'gamma_threshold': 0.001, 'random_state': None, 'minimum_probability': 0.01, 'minimum_phi_value': 0.01, 'per_word_topic': 0.01, 'update_every': 0}


,topic_terms,cv_coherence
topic_num,,
0,"0.004*""boil"" + 0.003*""salt"" + 0.003*""pound"" + 0.003*""butter"" + 0.003*""sugar"" + 0.003*""milk"" + 0.003*""pour"" + 0.002*""pint"" + 0.002*""cream"" + 0.002*""mix""",0.916451
38,"0.012*""thy"" + 0.011*""thou"" + 0.007*""unto"" + 0.005*""hath"" + 0.004*""ye"" + 0.004*""thee"" + 0.003*""christ"" + 0.002*""jesus"" + 0.002*""divine"" + 0.002*""hast""",0.803529
2,"0.008*""km"" + 0.005*""billion"" + 0.005*""na"" + 0.004*""million"" + 0.004*""population"" + 0.004*""military"" + 0.004*""national"" + 0.003*""islands"" + 0.003*""total"" + 0.003*""december""",0.742322
44,"0.005*""jesus"" + 0.003*""unto"" + 0.003*""christ"" + 0.003*""paul"" + 0.002*""thy"" + 0.002*""jewish"" + 0.002*""jews"" + 0.002*""ye"" + 0.001*""gospel"" + 0.001*""thou""",0.737823
45,"0.006*""dat"" + 0.006*""en"" + 0.005*""dey"" + 0.003*""ter"" + 0.003*""la"" + 0.003*""el"" + 0.002*""wid"" + 0.002*""que"" + 0.002*""git"" + 0.002*""wuz""",0.681790
17,"0.005*""british"" + 0.004*""german"" + 0.003*""national"" + 0.003*""president"" + 0.003*""military"" + 0.003*""troops"" + 0.003*""political"" + 0.002*""congress"" + 0.002*""officers"" + 0.002*""committee""",0.629701
14,"0.020*""sir"" + 0.002*""duke"" + 0.002*""earl"" + 0.002*""thou"" + 0.002*""knight"" + 0.002*""william"" + 0.002*""thy"" + 0.002*""queen"" + 0.002*""ye"" + 0.002*""henry""",0.620734
6,"0.010*""thou"" + 0.008*""thy"" + 0.004*""thee"" + 0.003*""ye"" + 0.002*""hath"" + 0.001*""hast"" + 0.001*""ere"" + 0.001*""prince"" + 0.001*""unto"" + 0.001*""bird""",0.618620
28,"0.002*""divine"" + 0.002*""spiritual"" + 0.002*""mental"" + 0.002*""science"" + 0.002*""relation"" + 0.002*""principle"" + 0.002*""conditions"" + 0.001*""principles"" + 0.001*""theory"" + 0.001*""hence""",0.601337


In [66]:
mod_2_df, mod_2_cv_coherence, mod_2_cv_topic_coherence, mod_2_run_params =\
    analyze_model_coherence('gpu_mod_run_2', corp_lst_stream)
    
print(mod_2_cv_coherence, mod_2_run_params)
mod_2_df

2018-03-28 12:53:37,421 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2.model
2018-03-28 12:53:37,440 : INFO : loading expElogbeta from ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2.model.expElogbeta.npy with mmap=None
2018-03-28 12:53:37,538 : INFO : setting ignored attribute dispatcher to None
2018-03-28 12:53:37,539 : INFO : setting ignored attribute state to None
2018-03-28 12:53:37,540 : INFO : setting ignored attribute id2word to None
2018-03-28 12:53:37,541 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2.model
2018-03-28 12:53:37,542 : INFO : dtype was not set in saved LdaMulticore file ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2.model, assuming np.float64
2018-03-28 12:53:37,544 : INFO : loading LdaState object from ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2.model.state
2018-03-28 12:53:37,558 : INFO : loading sstats from ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2.model.sta

corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2_mod_run_params.txt
pyLDAvis html fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2_ldavis.html
coherence model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2_coherence_model.
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2_mod_run_params.txt


2018-03-28 12:53:37,649 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2.model.state
2018-03-28 12:53:37,650 : INFO : dtype was not set in saved LdaState file ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2.model.state, assuming np.float64
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
2018-03-28 13:00:35,883 : INFO : saving CoherenceModel object under ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2_coherence_model.cv, separately None
2018-03-28 13:00:35,885 : INFO : storing np array 'sstats' to ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2_coherence_model.cv._model.state.sstats.npy
2018-03-28 13:00:35,948 : INFO : storing np array 'expElogbeta' to ../.

2018-03-28 13:03:02,783 : INFO : 62 batches submitted to accumulate stats from 3968 documents (35703571 virtual)
2018-03-28 13:03:09,273 : INFO : 63 batches submitted to accumulate stats from 4032 documents (36210981 virtual)
2018-03-28 13:03:10,596 : INFO : 64 batches submitted to accumulate stats from 4096 documents (36663722 virtual)
2018-03-28 13:03:11,251 : INFO : 65 batches submitted to accumulate stats from 4160 documents (37202399 virtual)
2018-03-28 13:03:12,419 : INFO : 66 batches submitted to accumulate stats from 4224 documents (37560385 virtual)
2018-03-28 13:03:13,928 : INFO : 67 batches submitted to accumulate stats from 4288 documents (38196793 virtual)
2018-03-28 13:03:17,030 : INFO : 68 batches submitted to accumulate stats from 4352 documents (38650823 virtual)
2018-03-28 13:03:18,373 : INFO : 69 batches submitted to accumulate stats from 4416 documents (39098416 virtual)
2018-03-28 13:03:24,397 : INFO : 70 batches submitted to accumulate stats from 4480 documents (3

2018-03-28 13:06:05,078 : INFO : 135 batches submitted to accumulate stats from 8640 documents (76260036 virtual)
2018-03-28 13:06:06,115 : INFO : 136 batches submitted to accumulate stats from 8704 documents (76750590 virtual)
2018-03-28 13:06:08,808 : INFO : 137 batches submitted to accumulate stats from 8768 documents (77411895 virtual)
2018-03-28 13:06:13,563 : INFO : 138 batches submitted to accumulate stats from 8832 documents (77996318 virtual)
2018-03-28 13:06:16,164 : INFO : 139 batches submitted to accumulate stats from 8896 documents (78525533 virtual)
2018-03-28 13:06:20,094 : INFO : 140 batches submitted to accumulate stats from 8960 documents (79115828 virtual)
2018-03-28 13:06:22,277 : INFO : 141 batches submitted to accumulate stats from 9024 documents (79690300 virtual)
2018-03-28 13:06:22,637 : INFO : 142 batches submitted to accumulate stats from 9088 documents (80250005 virtual)
2018-03-28 13:06:23,732 : INFO : 143 batches submitted to accumulate stats from 9152 doc

...

2018-03-28 13:11:28,386 : INFO : 278 batches submitted to accumulate stats from 17792 documents (146784799 virtual)
2018-03-28 13:11:30,996 : INFO : 279 batches submitted to accumulate stats from 17856 documents (147232904 virtual)
2018-03-28 13:11:32,019 : INFO : 280 batches submitted to accumulate stats from 17920 documents (147659055 virtual)
2018-03-28 13:11:32,868 : INFO : 281 batches submitted to accumulate stats from 17984 documents (148178321 virtual)
2018-03-28 13:11:34,874 : INFO : 282 batches submitted to accumulate stats from 18048 documents (148710972 virtual)
2018-03-28 13:11:36,009 : INFO : 283 batches submitted to accumulate stats from 18112 documents (149103562 virtual)
2018-03-28 13:11:38,160 : INFO : 284 batches submitted to accumulate stats from 18176 documents (149423778 virtual)
2018-03-28 13:11:43,303 : INFO : 285 batches submitted to accumulate stats from 18240 documents (149836218 virtual)
2018-03-28 13:11:45,205 : INFO : 286 batches submitted to accumulate sta

2018-03-28 13:14:10,324 : INFO : 349 batches submitted to accumulate stats from 22336 documents (185376177 virtual)
2018-03-28 13:14:13,779 : INFO : 350 batches submitted to accumulate stats from 22400 documents (185898641 virtual)
2018-03-28 13:14:16,622 : INFO : 351 batches submitted to accumulate stats from 22464 documents (186316095 virtual)
2018-03-28 13:14:17,298 : INFO : 352 batches submitted to accumulate stats from 22528 documents (186769649 virtual)
2018-03-28 13:14:19,259 : INFO : 353 batches submitted to accumulate stats from 22592 documents (187257660 virtual)
2018-03-28 13:14:23,740 : INFO : 354 batches submitted to accumulate stats from 22656 documents (187725695 virtual)
2018-03-28 13:14:29,137 : INFO : 355 batches submitted to accumulate stats from 22720 documents (188316870 virtual)
2018-03-28 13:14:30,577 : INFO : 356 batches submitted to accumulate stats from 22784 documents (188857737 virtual)
2018-03-28 13:14:34,927 : INFO : 357 batches submitted to accumulate sta

2018-03-28 13:17:20,336 : INFO : 420 batches submitted to accumulate stats from 26880 documents (228415227 virtual)
2018-03-28 13:17:23,703 : INFO : 421 batches submitted to accumulate stats from 26944 documents (228880671 virtual)
2018-03-28 13:17:24,803 : INFO : 422 batches submitted to accumulate stats from 27008 documents (229525568 virtual)
2018-03-28 13:17:25,313 : INFO : 423 batches submitted to accumulate stats from 27072 documents (230270271 virtual)
2018-03-28 13:17:32,189 : INFO : 424 batches submitted to accumulate stats from 27136 documents (230869983 virtual)
2018-03-28 13:17:34,529 : INFO : 425 batches submitted to accumulate stats from 27200 documents (231514702 virtual)
2018-03-28 13:17:36,691 : INFO : 426 batches submitted to accumulate stats from 27264 documents (232138940 virtual)
2018-03-28 13:17:38,601 : INFO : 427 batches submitted to accumulate stats from 27328 documents (232782381 virtual)
2018-03-28 13:17:42,082 : INFO : 428 batches submitted to accumulate sta

0.4652977169618066 {'num_topics': 50, 'cores': 41, 'chunksize': 2000, 'passes': 5, 'batch': False, 'alpha': 'symmetric', 'eta': None, 'decay': 0.5, 'offset': 1.0, 'eval_every': 10, 'iterations': 50, 'gamma_threshold': 0.001, 'random_state': None, 'minimum_probability': 0.01, 'minimum_phi_value': 0.01, 'per_word_topic': 0.01, 'update_every': 0, 'dim_reduction_run_used': 'dr_500k'}


,topic_terms,cv_coherence
topic_num,,
32,"0.003*""boil"" + 0.002*""eggs"" + 0.002*""salt"" + 0.002*""butter"" + 0.002*""pound"" + 0.002*""sugar"" + 0.002*""pour"" + 0.002*""uncle"" + 0.002*""cream"" + 0.002*""stir""",0.868293
1,"0.004*""dat"" + 0.004*""ter"" + 0.004*""dey"" + 0.003*""ye"" + 0.003*""en"" + 0.002*""aunt"" + 0.002*""git"" + 0.002*""wid"" + 0.002*""fer"" + 0.002*""uncle""",0.832853
49,"0.003*""troops"" + 0.003*""colonel"" + 0.002*""officers"" + 0.002*""soldiers"" + 0.002*""guns"" + 0.002*""division"" + 0.002*""regiment"" + 0.002*""cavalry"" + 0.002*""officer"" + 0.002*""brigade""",0.793089
27,"0.009*""km"" + 0.006*""billion"" + 0.006*""na"" + 0.005*""million"" + 0.004*""national"" + 0.004*""military"" + 0.004*""population"" + 0.004*""islands"" + 0.004*""december"" + 0.003*""total""",0.791467
7,"0.009*""unto"" + 0.009*""thou"" + 0.009*""thy"" + 0.006*""ye"" + 0.005*""jesus"" + 0.005*""hath"" + 0.005*""christ"" + 0.004*""divine"" + 0.003*""thee"" + 0.002*""spiritual""",0.746650
22,"0.013*""thou"" + 0.012*""thy"" + 0.005*""thee"" + 0.003*""hath"" + 0.003*""ye"" + 0.002*""sir"" + 0.002*""hast"" + 0.002*""queen"" + 0.002*""unto"" + 0.001*""prince""",0.684785
33,"0.005*""la"" + 0.004*""duke"" + 0.003*""et"" + 0.003*""le"" + 0.003*""queen"" + 0.003*""prince"" + 0.002*""charles"" + 0.002*""emperor"" + 0.002*""des"" + 0.002*""france""",0.677824
28,"0.003*""per"" + 0.002*""acid"" + 0.002*""solution"" + 0.002*""oil"" + 0.002*""pressure"" + 0.002*""temperature"" + 0.002*""steam"" + 0.002*""quantity"" + 0.001*""gas"" + 0.001*""inches""",0.658910
9,"0.003*""political"" + 0.003*""british"" + 0.003*""national"" + 0.002*""german"" + 0.002*""military"" + 0.002*""congress"" + 0.002*""president"" + 0.002*""religious"" + 0.001*""constitution"" + 0.001*""france""",0.655476


In [71]:
mod_3_df, mod_3_cv_coherence, mod_3_cv_topic_coherence, mod_3_run_params =\
    analyze_model_coherence('gpu_mod_run_3', corp_lst_stream)
    
print(mod_3_cv_coherence, mod_3_run_params)
mod_3_df

2018-03-28 14:32:57,060 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3.model
2018-03-28 14:32:57,075 : INFO : loading expElogbeta from ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3.model.expElogbeta.npy with mmap=None
2018-03-28 14:32:57,130 : INFO : setting ignored attribute dispatcher to None
2018-03-28 14:32:57,132 : INFO : setting ignored attribute state to None
2018-03-28 14:32:57,134 : INFO : setting ignored attribute id2word to None
2018-03-28 14:32:57,136 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3.model
2018-03-28 14:32:57,137 : INFO : dtype was not set in saved LdaMulticore file ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3.model, assuming np.float64
2018-03-28 14:32:57,139 : INFO : loading LdaState object from ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3.model.state


corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3_mod_run_params.txt
pyLDAvis html fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3_ldavis.html
coherence model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3_coherence_model.
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3_mod_run_params.txt


2018-03-28 14:32:57,633 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3.model.state
2018-03-28 14:32:57,635 : INFO : dtype was not set in saved LdaState file ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3.model.state, assuming np.float64
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
2018-03-28 14:37:52,465 : INFO : saving CoherenceModel object under ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3_coherence_model.cv, separately None
2018-03-28 14:37:54,012 : INFO : saved ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3_coherence_model.cv
2018-03-28 14:37:54,018 : INFO : using ParallelWordOccurrenceAccumulator(processes=7, batch_size=64) to estimate proba

2018-03-28 14:40:05,020 : INFO : 65 batches submitted to accumulate stats from 4160 documents (37205810 virtual)
2018-03-28 14:40:05,251 : INFO : 66 batches submitted to accumulate stats from 4224 documents (37561483 virtual)
2018-03-28 14:40:07,023 : INFO : 67 batches submitted to accumulate stats from 4288 documents (38210772 virtual)
2018-03-28 14:40:11,044 : INFO : 68 batches submitted to accumulate stats from 4352 documents (38652328 virtual)
2018-03-28 14:40:12,576 : INFO : 69 batches submitted to accumulate stats from 4416 documents (39110495 virtual)
2018-03-28 14:40:17,678 : INFO : 70 batches submitted to accumulate stats from 4480 documents (39648621 virtual)
2018-03-28 14:40:18,131 : INFO : 71 batches submitted to accumulate stats from 4544 documents (40279388 virtual)
2018-03-28 14:40:23,818 : INFO : 72 batches submitted to accumulate stats from 4608 documents (40793287 virtual)
2018-03-28 14:40:24,177 : INFO : 73 batches submitted to accumulate stats from 4672 documents (4

2018-03-28 14:42:48,070 : INFO : 138 batches submitted to accumulate stats from 8832 documents (77999717 virtual)
2018-03-28 14:42:50,540 : INFO : 139 batches submitted to accumulate stats from 8896 documents (78530311 virtual)
2018-03-28 14:42:54,832 : INFO : 140 batches submitted to accumulate stats from 8960 documents (79124977 virtual)
2018-03-28 14:42:56,181 : INFO : 141 batches submitted to accumulate stats from 9024 documents (79695713 virtual)
2018-03-28 14:42:56,911 : INFO : 142 batches submitted to accumulate stats from 9088 documents (80254766 virtual)
2018-03-28 14:42:57,368 : INFO : 143 batches submitted to accumulate stats from 9152 documents (80903576 virtual)
2018-03-28 14:42:58,672 : INFO : 144 batches submitted to accumulate stats from 9216 documents (81476734 virtual)
2018-03-28 14:43:06,303 : INFO : 145 batches submitted to accumulate stats from 9280 documents (82133397 virtual)
2018-03-28 14:43:07,214 : INFO : 146 batches submitted to accumulate stats from 9344 doc

...

2018-03-28 14:47:35,586 : INFO : 281 batches submitted to accumulate stats from 17984 documents (148218137 virtual)
2018-03-28 14:47:38,444 : INFO : 282 batches submitted to accumulate stats from 18048 documents (148764608 virtual)
2018-03-28 14:47:38,679 : INFO : 283 batches submitted to accumulate stats from 18112 documents (149116468 virtual)
2018-03-28 14:47:39,010 : INFO : 284 batches submitted to accumulate stats from 18176 documents (149493413 virtual)
2018-03-28 14:47:44,089 : INFO : 285 batches submitted to accumulate stats from 18240 documents (149882584 virtual)
2018-03-28 14:47:45,789 : INFO : 286 batches submitted to accumulate stats from 18304 documents (150310921 virtual)
2018-03-28 14:47:46,804 : INFO : 287 batches submitted to accumulate stats from 18368 documents (150636123 virtual)
2018-03-28 14:47:48,951 : INFO : 288 batches submitted to accumulate stats from 18432 documents (151156589 virtual)
2018-03-28 14:47:49,855 : INFO : 289 batches submitted to accumulate sta

2018-03-28 14:50:01,939 : INFO : 352 batches submitted to accumulate stats from 22528 documents (186834449 virtual)
2018-03-28 14:50:02,501 : INFO : 353 batches submitted to accumulate stats from 22592 documents (187313847 virtual)
2018-03-28 14:50:06,596 : INFO : 354 batches submitted to accumulate stats from 22656 documents (187782848 virtual)
2018-03-28 14:50:11,168 : INFO : 355 batches submitted to accumulate stats from 22720 documents (188366342 virtual)
2018-03-28 14:50:12,261 : INFO : 356 batches submitted to accumulate stats from 22784 documents (188908503 virtual)
2018-03-28 14:50:16,227 : INFO : 357 batches submitted to accumulate stats from 22848 documents (189394598 virtual)
2018-03-28 14:50:18,598 : INFO : 358 batches submitted to accumulate stats from 22912 documents (190051860 virtual)
2018-03-28 14:50:19,563 : INFO : 359 batches submitted to accumulate stats from 22976 documents (190499622 virtual)
2018-03-28 14:50:21,170 : INFO : 360 batches submitted to accumulate sta

2018-03-28 14:52:50,669 : INFO : 423 batches submitted to accumulate stats from 27072 documents (230329083 virtual)
2018-03-28 14:52:59,966 : INFO : 424 batches submitted to accumulate stats from 27136 documents (230995513 virtual)
2018-03-28 14:53:00,333 : INFO : 425 batches submitted to accumulate stats from 27200 documents (231556984 virtual)
2018-03-28 14:53:01,594 : INFO : 426 batches submitted to accumulate stats from 27264 documents (232178205 virtual)
2018-03-28 14:53:04,851 : INFO : 427 batches submitted to accumulate stats from 27328 documents (232851488 virtual)
2018-03-28 14:53:05,966 : INFO : 428 batches submitted to accumulate stats from 27392 documents (233684381 virtual)
2018-03-28 14:53:06,455 : INFO : 429 batches submitted to accumulate stats from 27456 documents (234261498 virtual)
2018-03-28 14:53:14,734 : INFO : 430 batches submitted to accumulate stats from 27520 documents (235023215 virtual)
2018-03-28 14:53:15,116 : INFO : 431 batches submitted to accumulate sta

0.46248222267854033 {'num_topics': 30, 'cores': 41, 'chunksize': 2000, 'passes': 5, 'batch': False, 'alpha': 'symmetric', 'eta': None, 'decay': 0.5, 'offset': 1.0, 'eval_every': 10, 'iterations': 50, 'gamma_threshold': 0.001, 'random_state': None, 'minimum_probability': 0.01, 'minimum_phi_value': 0.01, 'per_word_topic': 0.01, 'update_every': 0, 'dim_reduction_run_used': 'dr_500k'}


,topic_terms,cv_coherence
topic_num,,
8,"0.007*""km"" + 0.004*""billion"" + 0.004*""na"" + 0.004*""million"" + 0.004*""national"" + 0.003*""population"" + 0.003*""military"" + 0.003*""total"" + 0.003*""president"" + 0.003*""islands""",0.783195
2,"0.015*""thou"" + 0.013*""thy"" + 0.006*""unto"" + 0.005*""thee"" + 0.005*""hath"" + 0.004*""ye"" + 0.003*""jesus"" + 0.002*""hast"" + 0.002*""christ"" + 0.002*""divine""",0.733490
6,"0.005*""german"" + 0.004*""british"" + 0.002*""officers"" + 0.002*""troops"" + 0.002*""germans"" + 0.002*""guns"" + 0.002*""sir"" + 0.002*""military"" + 0.001*""officer"" + 0.001*""russian""",0.651828
27,"0.003*""political"" + 0.002*""national"" + 0.002*""roman"" + 0.002*""congress"" + 0.002*""president"" + 0.002*""constitution"" + 0.001*""british"" + 0.001*""military"" + 0.001*""rights"" + 0.001*""bill""",0.635148
20,"0.004*""indians"" + 0.004*""indian"" + 0.003*""fort"" + 0.002*""colonel"" + 0.002*""british"" + 0.002*""officers"" + 0.002*""camp"" + 0.002*""sir"" + 0.002*""troops"" + 0.001*""lake""",0.615753
1,"0.002*""ships"" + 0.002*""boat"" + 0.002*""island"" + 0.001*""british"" + 0.001*""sir"" + 0.001*""coast"" + 0.001*""cape"" + 0.001*""vessel"" + 0.001*""thou"" + 0.001*""natives""",0.606940
16,"0.004*""la"" + 0.003*""et"" + 0.002*""duke"" + 0.002*""charles"" + 0.002*""sir"" + 0.002*""le"" + 0.002*""prince"" + 0.002*""france"" + 0.002*""queen"" + 0.001*""henry""",0.603991
0,"0.001*""mental"" + 0.001*""science"" + 0.001*""sir"" + 0.001*""spiritual"" + 0.001*""religious"" + 0.001*""religion"" + 0.001*""literary"" + 0.001*""divine"" + 0.001*""intellectual"" + 0.001*""relation""",0.579982
15,"0.003*""british"" + 0.002*""chinese"" + 0.002*""president"" + 0.002*""military"" + 0.002*""spanish"" + 0.002*""german"" + 0.001*""congress"" + 0.001*""national"" + 0.001*""island"" + 0.001*""per""",0.536618


In [70]:
mod_4_df, mod_4_cv_coherence, mod_4_cv_topic_coherence, mod_4_run_params =\
    analyze_model_coherence('gpu_mod_run_4', corp_lst_stream)
    
print(mod_4_cv_coherence, mod_4_run_params)
mod_4_df

2018-03-28 14:11:50,246 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model
2018-03-28 14:11:50,273 : INFO : loading expElogbeta from ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model.expElogbeta.npy with mmap=None
2018-03-28 14:11:50,318 : INFO : setting ignored attribute id2word to None
2018-03-28 14:11:50,320 : INFO : setting ignored attribute state to None
2018-03-28 14:11:50,322 : INFO : setting ignored attribute dispatcher to None
2018-03-28 14:11:50,323 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model
2018-03-28 14:11:50,326 : INFO : dtype was not set in saved LdaMulticore file ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model, assuming np.float64
2018-03-28 14:11:50,327 : INFO : loading LdaState object from ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model.state


corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_mod_run_params.txt
pyLDAvis html fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_ldavis.html
coherence model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_coherence_model.
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_mod_run_params.txt


2018-03-28 14:11:50,666 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model.state
2018-03-28 14:11:50,669 : INFO : dtype was not set in saved LdaState file ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model.state, assuming np.float64
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
2018-03-28 14:15:59,895 : INFO : saving CoherenceModel object under ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_coherence_model.cv, separately None
2018-03-28 14:16:00,959 : INFO : saved ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_coherence_model.cv
2018-03-28 14:16:00,964 : INFO : using ParallelWordOccurrenceAccumulator(processes=7, batch_size=64) to estimate proba

2018-03-28 14:18:18,556 : INFO : 65 batches submitted to accumulate stats from 4160 documents (37202399 virtual)
2018-03-28 14:18:18,766 : INFO : 66 batches submitted to accumulate stats from 4224 documents (37560385 virtual)
2018-03-28 14:18:21,126 : INFO : 67 batches submitted to accumulate stats from 4288 documents (38196793 virtual)
2018-03-28 14:18:23,271 : INFO : 68 batches submitted to accumulate stats from 4352 documents (38652126 virtual)
2018-03-28 14:18:26,119 : INFO : 69 batches submitted to accumulate stats from 4416 documents (39110293 virtual)
2018-03-28 14:18:30,528 : INFO : 70 batches submitted to accumulate stats from 4480 documents (39648419 virtual)
2018-03-28 14:18:31,664 : INFO : 71 batches submitted to accumulate stats from 4544 documents (40279186 virtual)
2018-03-28 14:18:38,105 : INFO : 72 batches submitted to accumulate stats from 4608 documents (40793085 virtual)
2018-03-28 14:18:39,010 : INFO : 73 batches submitted to accumulate stats from 4672 documents (4

2018-03-28 14:21:06,516 : INFO : 138 batches submitted to accumulate stats from 8832 documents (77999515 virtual)
2018-03-28 14:21:09,660 : INFO : 139 batches submitted to accumulate stats from 8896 documents (78530109 virtual)
2018-03-28 14:21:13,952 : INFO : 140 batches submitted to accumulate stats from 8960 documents (79124775 virtual)
2018-03-28 14:21:14,856 : INFO : 141 batches submitted to accumulate stats from 9024 documents (79695511 virtual)
2018-03-28 14:21:15,543 : INFO : 142 batches submitted to accumulate stats from 9088 documents (80254564 virtual)
2018-03-28 14:21:16,718 : INFO : 143 batches submitted to accumulate stats from 9152 documents (80903374 virtual)
2018-03-28 14:21:17,070 : INFO : 144 batches submitted to accumulate stats from 9216 documents (81476532 virtual)
2018-03-28 14:21:25,942 : INFO : 145 batches submitted to accumulate stats from 9280 documents (82133195 virtual)
2018-03-28 14:21:26,728 : INFO : 146 batches submitted to accumulate stats from 9344 doc

...

2018-03-28 14:26:06,252 : INFO : 281 batches submitted to accumulate stats from 17984 documents (148181663 virtual)
2018-03-28 14:26:07,558 : INFO : 282 batches submitted to accumulate stats from 18048 documents (148737749 virtual)
2018-03-28 14:26:07,795 : INFO : 283 batches submitted to accumulate stats from 18112 documents (149111128 virtual)
2018-03-28 14:26:10,220 : INFO : 284 batches submitted to accumulate stats from 18176 documents (149436808 virtual)
2018-03-28 14:26:14,099 : INFO : 285 batches submitted to accumulate stats from 18240 documents (149843533 virtual)
2018-03-28 14:26:15,896 : INFO : 286 batches submitted to accumulate stats from 18304 documents (150294597 virtual)
2018-03-28 14:26:18,107 : INFO : 287 batches submitted to accumulate stats from 18368 documents (150620000 virtual)
2018-03-28 14:26:20,399 : INFO : 288 batches submitted to accumulate stats from 18432 documents (151103828 virtual)
2018-03-28 14:26:20,638 : INFO : 289 batches submitted to accumulate sta

2018-03-28 14:28:35,578 : INFO : 352 batches submitted to accumulate stats from 22528 documents (186793282 virtual)
2018-03-28 14:28:37,302 : INFO : 353 batches submitted to accumulate stats from 22592 documents (187270262 virtual)
2018-03-28 14:28:40,856 : INFO : 354 batches submitted to accumulate stats from 22656 documents (187734468 virtual)
2018-03-28 14:28:45,627 : INFO : 355 batches submitted to accumulate stats from 22720 documents (188332002 virtual)
2018-03-28 14:28:48,052 : INFO : 356 batches submitted to accumulate stats from 22784 documents (188874893 virtual)
2018-03-28 14:28:49,866 : INFO : 357 batches submitted to accumulate stats from 22848 documents (189335232 virtual)
2018-03-28 14:28:53,002 : INFO : 358 batches submitted to accumulate stats from 22912 documents (189998909 virtual)
2018-03-28 14:28:54,104 : INFO : 359 batches submitted to accumulate stats from 22976 documents (190452972 virtual)
2018-03-28 14:28:55,093 : INFO : 360 batches submitted to accumulate sta

2018-03-28 14:31:34,818 : INFO : 423 batches submitted to accumulate stats from 27072 documents (230295821 virtual)
2018-03-28 14:31:42,525 : INFO : 424 batches submitted to accumulate stats from 27136 documents (230932605 virtual)
2018-03-28 14:31:44,629 : INFO : 425 batches submitted to accumulate stats from 27200 documents (231538568 virtual)
2018-03-28 14:31:45,061 : INFO : 426 batches submitted to accumulate stats from 27264 documents (232165409 virtual)
2018-03-28 14:31:49,938 : INFO : 427 batches submitted to accumulate stats from 27328 documents (232822306 virtual)
2018-03-28 14:31:50,589 : INFO : 428 batches submitted to accumulate stats from 27392 documents (233639502 virtual)
2018-03-28 14:31:51,057 : INFO : 429 batches submitted to accumulate stats from 27456 documents (234231618 virtual)
2018-03-28 14:31:59,119 : INFO : 430 batches submitted to accumulate stats from 27520 documents (234989616 virtual)
2018-03-28 14:31:59,679 : INFO : 431 batches submitted to accumulate sta

0.5797411828978578 {'num_topics': 20, 'cores': 9, 'chunksize': 2000, 'passes': 5, 'batch': False, 'alpha': 'symmetric', 'eta': None, 'decay': 0.5, 'offset': 1.0, 'eval_every': 10, 'iterations': 50, 'gamma_threshold': 0.001, 'random_state': None, 'minimum_probability': 0.01, 'minimum_phi_value': 0.01, 'per_word_topic': 0.01, 'update_every': 0, 'dim_reduction_run_used': 'dr_500k'}


,topic_terms,cv_coherence
topic_num,,
18,"0.003*""salt"" + 0.003*""boil"" + 0.003*""milk"" + 0.003*""sugar"" + 0.002*""butter"" + 0.002*""pound"" + 0.002*""pour"" + 0.002*""mix"" + 0.002*""cream"" + 0.002*""boiling""",0.905420
6,"0.009*""km"" + 0.006*""billion"" + 0.006*""na"" + 0.005*""million"" + 0.004*""december"" + 0.004*""population"" + 0.004*""military"" + 0.004*""national"" + 0.004*""islands"" + 0.004*""comparison""",0.790977
9,"0.010*""la"" + 0.004*""et"" + 0.004*""le"" + 0.004*""que"" + 0.003*""en"" + 0.003*""les"" + 0.003*""des"" + 0.002*""du"" + 0.002*""el"" + 0.002*""sir""",0.755164
15,"0.016*""thou"" + 0.015*""thy"" + 0.006*""thee"" + 0.005*""unto"" + 0.004*""ye"" + 0.004*""hath"" + 0.002*""hast"" + 0.002*""shalt"" + 0.001*""thine"" + 0.001*""prince""",0.735012
7,"0.005*""british"" + 0.004*""german"" + 0.004*""troops"" + 0.003*""officers"" + 0.003*""colonel"" + 0.003*""fort"" + 0.002*""military"" + 0.002*""guns"" + 0.002*""officer"" + 0.002*""soldiers""",0.670527
17,"0.003*""indian"" + 0.003*""indians"" + 0.002*""island"" + 0.002*""lake"" + 0.002*""boat"" + 0.002*""natives"" + 0.002*""coast"" + 0.001*""cape"" + 0.001*""ice"" + 0.001*""spanish""",0.651971
10,"0.003*""political"" + 0.003*""national"" + 0.003*""president"" + 0.002*""congress"" + 0.002*""british"" + 0.002*""constitution"" + 0.002*""per"" + 0.002*""committee"" + 0.002*""rights"" + 0.002*""william""",0.647325
16,"0.003*""madame"" + 0.002*""la"" + 0.002*""monsieur"" + 0.001*""paris"" + 0.001*""prince"" + 0.001*""literary"" + 0.001*""louis"" + 0.001*""artist"" + 0.001*""pictures"" + 0.001*""italian""",0.585713
11,"0.012*""sir"" + 0.003*""duke"" + 0.002*""queen"" + 0.002*""henry"" + 0.002*""william"" + 0.002*""earl"" + 0.002*""charles"" + 0.002*""prince"" + 0.001*""mary"" + 0.001*""edward""",0.574628


In [68]:
mod_5_df, mod_5_cv_coherence, mod_5_cv_topic_coherence, mod_5_run_params =\
    analyze_model_coherence('gpu_mod_run_5', corp_lst_stream)
    
print(mod_5_cv_coherence, mod_5_run_params)
mod_5_df

2018-03-28 13:32:22,319 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5.model
2018-03-28 13:32:22,341 : INFO : loading expElogbeta from ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5.model.expElogbeta.npy with mmap=None
2018-03-28 13:32:22,385 : INFO : setting ignored attribute state to None
2018-03-28 13:32:22,387 : INFO : setting ignored attribute dispatcher to None
2018-03-28 13:32:22,389 : INFO : setting ignored attribute id2word to None
2018-03-28 13:32:22,391 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5.model
2018-03-28 13:32:22,393 : INFO : dtype was not set in saved LdaMulticore file ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5.model, assuming np.float64
2018-03-28 13:32:22,395 : INFO : loading LdaState object from ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5.model.state


corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5_mod_run_params.txt
pyLDAvis html fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5_ldavis.html
coherence model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5_coherence_model.
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5_mod_run_params.txt


2018-03-28 13:32:22,737 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5.model.state
2018-03-28 13:32:22,739 : INFO : dtype was not set in saved LdaState file ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5.model.state, assuming np.float64
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
2018-03-28 13:36:25,400 : INFO : saving CoherenceModel object under ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5_coherence_model.cv, separately None
2018-03-28 13:36:26,430 : INFO : saved ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5_coherence_model.cv
2018-03-28 13:36:26,434 : INFO : using ParallelWordOccurrenceAccumulator(processes=7, batch_size=64) to estimate proba

2018-03-28 13:37:53,536 : INFO : 65 batches submitted to accumulate stats from 4160 documents (37364831 virtual)
2018-03-28 13:37:55,571 : INFO : 66 batches submitted to accumulate stats from 4224 documents (37958048 virtual)
2018-03-28 13:37:56,953 : INFO : 67 batches submitted to accumulate stats from 4288 documents (38470413 virtual)
2018-03-28 13:37:57,637 : INFO : 68 batches submitted to accumulate stats from 4352 documents (38902475 virtual)
2018-03-28 13:37:57,988 : INFO : 69 batches submitted to accumulate stats from 4416 documents (39445667 virtual)
2018-03-28 13:38:00,305 : INFO : 70 batches submitted to accumulate stats from 4480 documents (40060595 virtual)
2018-03-28 13:38:03,810 : INFO : 71 batches submitted to accumulate stats from 4544 documents (40555323 virtual)
2018-03-28 13:38:05,963 : INFO : 72 batches submitted to accumulate stats from 4608 documents (41005684 virtual)
2018-03-28 13:38:07,093 : INFO : 73 batches submitted to accumulate stats from 4672 documents (4

2018-03-28 13:39:41,218 : INFO : 138 batches submitted to accumulate stats from 8832 documents (78417619 virtual)
2018-03-28 13:39:46,404 : INFO : 139 batches submitted to accumulate stats from 8896 documents (78994697 virtual)
2018-03-28 13:39:46,746 : INFO : 140 batches submitted to accumulate stats from 8960 documents (79558792 virtual)
2018-03-28 13:39:47,102 : INFO : 141 batches submitted to accumulate stats from 9024 documents (80145466 virtual)
2018-03-28 13:39:47,544 : INFO : 142 batches submitted to accumulate stats from 9088 documents (80790805 virtual)
2018-03-28 13:39:47,895 : INFO : 143 batches submitted to accumulate stats from 9152 documents (81375265 virtual)
2018-03-28 13:39:52,585 : INFO : 144 batches submitted to accumulate stats from 9216 documents (82002479 virtual)
2018-03-28 13:39:53,990 : INFO : 145 batches submitted to accumulate stats from 9280 documents (82571333 virtual)
2018-03-28 13:39:54,597 : INFO : 146 batches submitted to accumulate stats from 9344 doc

...

2018-03-28 13:42:48,477 : INFO : 281 batches submitted to accumulate stats from 17984 documents (149481424 virtual)
2018-03-28 13:42:51,457 : INFO : 282 batches submitted to accumulate stats from 18048 documents (149910142 virtual)
2018-03-28 13:42:53,386 : INFO : 283 batches submitted to accumulate stats from 18112 documents (150358759 virtual)
2018-03-28 13:42:53,920 : INFO : 284 batches submitted to accumulate stats from 18176 documents (150734201 virtual)
2018-03-28 13:42:55,084 : INFO : 285 batches submitted to accumulate stats from 18240 documents (151237547 virtual)
2018-03-28 13:42:55,789 : INFO : 286 batches submitted to accumulate stats from 18304 documents (151631040 virtual)
2018-03-28 13:42:57,491 : INFO : 287 batches submitted to accumulate stats from 18368 documents (152190415 virtual)
2018-03-28 13:42:58,841 : INFO : 288 batches submitted to accumulate stats from 18432 documents (152827682 virtual)
2018-03-28 13:42:59,141 : INFO : 289 batches submitted to accumulate sta

2018-03-28 13:44:28,884 : INFO : 352 batches submitted to accumulate stats from 22528 documents (188831347 virtual)
2018-03-28 13:44:32,208 : INFO : 353 batches submitted to accumulate stats from 22592 documents (189348287 virtual)
2018-03-28 13:44:32,842 : INFO : 354 batches submitted to accumulate stats from 22656 documents (190036782 virtual)
2018-03-28 13:44:33,077 : INFO : 355 batches submitted to accumulate stats from 22720 documents (190454738 virtual)
2018-03-28 13:44:33,975 : INFO : 356 batches submitted to accumulate stats from 22784 documents (190930460 virtual)
2018-03-28 13:44:36,426 : INFO : 357 batches submitted to accumulate stats from 22848 documents (191451002 virtual)
2018-03-28 13:44:39,754 : INFO : 358 batches submitted to accumulate stats from 22912 documents (191989085 virtual)
2018-03-28 13:44:40,403 : INFO : 359 batches submitted to accumulate stats from 22976 documents (192514694 virtual)
2018-03-28 13:44:40,791 : INFO : 360 batches submitted to accumulate sta

2018-03-28 13:46:23,539 : INFO : 423 batches submitted to accumulate stats from 27072 documents (232961540 virtual)
2018-03-28 13:46:24,268 : INFO : 424 batches submitted to accumulate stats from 27136 documents (233825565 virtual)
2018-03-28 13:46:24,830 : INFO : 425 batches submitted to accumulate stats from 27200 documents (234385444 virtual)
2018-03-28 13:46:28,866 : INFO : 426 batches submitted to accumulate stats from 27264 documents (235106798 virtual)
2018-03-28 13:46:29,290 : INFO : 427 batches submitted to accumulate stats from 27328 documents (235800197 virtual)
2018-03-28 13:46:31,714 : INFO : 428 batches submitted to accumulate stats from 27392 documents (236526516 virtual)
2018-03-28 13:46:32,494 : INFO : 429 batches submitted to accumulate stats from 27456 documents (236724840 virtual)
2018-03-28 13:46:44,897 : INFO : serializing accumulator to return to master...
2018-03-28 13:46:44,902 : INFO : accumulator serialized
2018-03-28 13:46:46,829 : INFO : serializing accumul

0.28193087808174444 {'num_topics': 20, 'cores': 9, 'chunksize': 2000, 'passes': 1, 'batch': False, 'alpha': 'symmetric', 'eta': None, 'decay': 0.5, 'offset': 1.0, 'eval_every': 10, 'iterations': 50, 'gamma_threshold': 0.001, 'random_state': None, 'minimum_probability': 0.01, 'minimum_phi_value': 0.01, 'per_word_topic': 0.01, 'update_every': 0, 'dim_reduction_run_used': 'dr_500k'}


,topic_terms,cv_coherence
topic_num,,
8,"0.002*""sir"" + 0.001*""thy"" + 0.001*""british"" + 0.001*""political"" + 0.001*""henry"" + 0.001*""mr"" + 0.001*""thou"" + 0.001*""la"" + 0.001*""tom"" + 0.001*""national""",0.363355
10,"0.002*""sir"" + 0.002*""thy"" + 0.001*""ye"" + 0.001*""thou"" + 0.001*""william"" + 0.001*""mary"" + 0.001*""prince"" + 0.001*""hath"" + 0.001*""thee"" + 0.001*""duke""",0.343554
0,"0.007*""thou"" + 0.005*""thy"" + 0.002*""unto"" + 0.002*""ye"" + 0.002*""sir"" + 0.002*""thee"" + 0.002*""hath"" + 0.001*""indian"" + 0.001*""hast"" + 0.001*""british""",0.333089
9,"0.002*""thy"" + 0.002*""sir"" + 0.001*""thou"" + 0.001*""unto"" + 0.001*""prince"" + 0.001*""ye"" + 0.001*""charles"" + 0.001*""religious"" + 0.001*""british"" + 0.001*""german""",0.330761
12,"0.002*""sir"" + 0.001*""thy"" + 0.001*""thou"" + 0.001*""british"" + 0.001*""ye"" + 0.001*""la"" + 0.001*""prince"" + 0.001*""dick"" + 0.001*""island"" + 0.001*""henry""",0.316167
5,"0.002*""sir"" + 0.002*""thou"" + 0.001*""thy"" + 0.001*""ye"" + 0.001*""la"" + 0.001*""british"" + 0.001*""indian"" + 0.001*""william"" + 0.001*""et"" + 0.001*""national""",0.309251
16,"0.003*""sir"" + 0.001*""ye"" + 0.001*""thou"" + 0.001*""thy"" + 0.001*""british"" + 0.001*""political"" + 0.001*""la"" + 0.001*""national"" + 0.001*""military"" + 0.001*""henry""",0.299105
14,"0.003*""thou"" + 0.003*""thy"" + 0.001*""sir"" + 0.001*""national"" + 0.001*""thee"" + 0.001*""hath"" + 0.001*""william"" + 0.001*""political"" + 0.001*""madame"" + 0.001*""henry""",0.297100
18,"0.003*""thy"" + 0.002*""sir"" + 0.002*""thou"" + 0.001*""ye"" + 0.001*""unto"" + 0.001*""british"" + 0.001*""prince"" + 0.001*""william"" + 0.001*""la"" + 0.001*""political""",0.289054


In [69]:
mod_6_df, mod_6_cv_coherence, mod_6_cv_topic_coherence, mod_6_run_params =\
    analyze_model_coherence('gpu_mod_run_6', corp_lst_stream)
    
print(mod_6_cv_coherence, mod_6_run_params)
mod_6_df

2018-03-28 13:48:06,729 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6.model
2018-03-28 13:48:06,751 : INFO : loading expElogbeta from ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6.model.expElogbeta.npy with mmap=None
2018-03-28 13:48:06,839 : INFO : setting ignored attribute dispatcher to None
2018-03-28 13:48:06,840 : INFO : setting ignored attribute state to None
2018-03-28 13:48:06,842 : INFO : setting ignored attribute id2word to None
2018-03-28 13:48:06,844 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6.model
2018-03-28 13:48:06,845 : INFO : dtype was not set in saved LdaMulticore file ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6.model, assuming np.float64
2018-03-28 13:48:06,846 : INFO : loading LdaState object from ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6.model.state
2018-03-28 13:48:06,865 : INFO : loading sstats from ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6.model.sta

corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6_mod_run_params.txt
pyLDAvis html fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6_ldavis.html
coherence model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6_coherence_model.
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6_mod_run_params.txt


2018-03-28 13:48:06,967 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6.model.state
2018-03-28 13:48:06,969 : INFO : dtype was not set in saved LdaState file ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6.model.state, assuming np.float64
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
2018-03-28 13:53:57,472 : INFO : saving CoherenceModel object under ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6_coherence_model.cv, separately None
2018-03-28 13:53:57,474 : INFO : storing np array 'sstats' to ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6_coherence_model.cv._model.state.sstats.npy
2018-03-28 13:53:57,536 : INFO : storing np array 'expElogbeta' to ../.

2018-03-28 13:56:07,359 : INFO : 62 batches submitted to accumulate stats from 3968 documents (35707072 virtual)
2018-03-28 13:56:14,332 : INFO : 63 batches submitted to accumulate stats from 4032 documents (36242271 virtual)
2018-03-28 13:56:15,061 : INFO : 64 batches submitted to accumulate stats from 4096 documents (36696244 virtual)
2018-03-28 13:56:15,442 : INFO : 65 batches submitted to accumulate stats from 4160 documents (37212121 virtual)
2018-03-28 13:56:17,181 : INFO : 66 batches submitted to accumulate stats from 4224 documents (37567712 virtual)
2018-03-28 13:56:19,818 : INFO : 67 batches submitted to accumulate stats from 4288 documents (38228887 virtual)
2018-03-28 13:56:20,123 : INFO : 68 batches submitted to accumulate stats from 4352 documents (38666029 virtual)
2018-03-28 13:56:24,109 : INFO : 69 batches submitted to accumulate stats from 4416 documents (39128634 virtual)
2018-03-28 13:56:29,707 : INFO : 70 batches submitted to accumulate stats from 4480 documents (3

2018-03-28 13:59:03,402 : INFO : 135 batches submitted to accumulate stats from 8640 documents (76299538 virtual)
2018-03-28 13:59:04,716 : INFO : 136 batches submitted to accumulate stats from 8704 documents (76859922 virtual)
2018-03-28 13:59:06,122 : INFO : 137 batches submitted to accumulate stats from 8768 documents (77472467 virtual)
2018-03-28 13:59:10,433 : INFO : 138 batches submitted to accumulate stats from 8832 documents (78039945 virtual)
2018-03-28 13:59:16,183 : INFO : 139 batches submitted to accumulate stats from 8896 documents (78565069 virtual)
2018-03-28 13:59:16,957 : INFO : 140 batches submitted to accumulate stats from 8960 documents (79185041 virtual)
2018-03-28 13:59:17,608 : INFO : 141 batches submitted to accumulate stats from 9024 documents (79725183 virtual)
2018-03-28 13:59:19,900 : INFO : 142 batches submitted to accumulate stats from 9088 documents (80299282 virtual)
2018-03-28 13:59:20,814 : INFO : 143 batches submitted to accumulate stats from 9152 doc

...

2018-03-28 14:04:22,756 : INFO : 278 batches submitted to accumulate stats from 17792 documents (146813138 virtual)
2018-03-28 14:04:26,689 : INFO : 279 batches submitted to accumulate stats from 17856 documents (147255983 virtual)
2018-03-28 14:04:27,139 : INFO : 280 batches submitted to accumulate stats from 17920 documents (147720006 virtual)
2018-03-28 14:04:28,971 : INFO : 281 batches submitted to accumulate stats from 17984 documents (148213208 virtual)
2018-03-28 14:04:29,340 : INFO : 282 batches submitted to accumulate stats from 18048 documents (148759718 virtual)
2018-03-28 14:04:29,824 : INFO : 283 batches submitted to accumulate stats from 18112 documents (149114867 virtual)
2018-03-28 14:04:31,794 : INFO : 284 batches submitted to accumulate stats from 18176 documents (149470377 virtual)
2018-03-28 14:04:37,392 : INFO : 285 batches submitted to accumulate stats from 18240 documents (149880160 virtual)
2018-03-28 14:04:39,084 : INFO : 286 batches submitted to accumulate sta

2018-03-28 14:07:00,372 : INFO : 349 batches submitted to accumulate stats from 22336 documents (185444816 virtual)
2018-03-28 14:07:02,135 : INFO : 350 batches submitted to accumulate stats from 22400 documents (185948205 virtual)
2018-03-28 14:07:05,850 : INFO : 351 batches submitted to accumulate stats from 22464 documents (186358870 virtual)
2018-03-28 14:07:08,335 : INFO : 352 batches submitted to accumulate stats from 22528 documents (186834009 virtual)
2018-03-28 14:07:08,782 : INFO : 353 batches submitted to accumulate stats from 22592 documents (187313407 virtual)
2018-03-28 14:07:12,123 : INFO : 354 batches submitted to accumulate stats from 22656 documents (187782408 virtual)
2018-03-28 14:07:18,298 : INFO : 355 batches submitted to accumulate stats from 22720 documents (188365902 virtual)
2018-03-28 14:07:18,642 : INFO : 356 batches submitted to accumulate stats from 22784 documents (188908063 virtual)
2018-03-28 14:07:24,131 : INFO : 357 batches submitted to accumulate sta

2018-03-28 14:10:10,987 : INFO : 420 batches submitted to accumulate stats from 26880 documents (228509610 virtual)
2018-03-28 14:10:12,087 : INFO : 421 batches submitted to accumulate stats from 26944 documents (228928793 virtual)
2018-03-28 14:10:13,291 : INFO : 422 batches submitted to accumulate stats from 27008 documents (229656749 virtual)
2018-03-28 14:10:13,832 : INFO : 423 batches submitted to accumulate stats from 27072 documents (230319433 virtual)
2018-03-28 14:10:23,384 : INFO : 424 batches submitted to accumulate stats from 27136 documents (230981106 virtual)
2018-03-28 14:10:23,773 : INFO : 425 batches submitted to accumulate stats from 27200 documents (231552066 virtual)
2018-03-28 14:10:25,170 : INFO : 426 batches submitted to accumulate stats from 27264 documents (232175328 virtual)
2018-03-28 14:10:29,707 : INFO : 427 batches submitted to accumulate stats from 27328 documents (232850514 virtual)
2018-03-28 14:10:30,666 : INFO : 428 batches submitted to accumulate sta

0.4429589860842077 {'num_topics': 40, 'cores': 41, 'chunksize': 2000, 'passes': 5, 'batch': False, 'alpha': 'symmetric', 'eta': None, 'decay': 0.5, 'offset': 1.0, 'eval_every': 10, 'iterations': 50, 'gamma_threshold': 0.001, 'random_state': None, 'minimum_probability': 0.01, 'minimum_phi_value': 0.01, 'per_word_topic': 0.01, 'update_every': 0, 'dim_reduction_run_used': 'dr_500k'}


,topic_terms,cv_coherence
topic_num,,
28,"0.002*""salt"" + 0.002*""sugar"" + 0.002*""boil"" + 0.002*""milk"" + 0.002*""eggs"" + 0.002*""butter"" + 0.002*""fruit"" + 0.002*""per"" + 0.002*""pound"" + 0.002*""pour""",0.790475
27,"0.010*""thou"" + 0.009*""thy"" + 0.009*""unto"" + 0.006*""ye"" + 0.005*""jesus"" + 0.005*""hath"" + 0.004*""christ"" + 0.003*""thee"" + 0.003*""divine"" + 0.002*""sin""",0.778404
4,"0.013*""thou"" + 0.013*""thy"" + 0.005*""thee"" + 0.003*""hath"" + 0.002*""ye"" + 0.002*""hast"" + 0.002*""sir"" + 0.001*""ere"" + 0.001*""song"" + 0.001*""doth""",0.755204
19,"0.004*""president"" + 0.003*""lincoln"" + 0.002*""congress"" + 0.002*""secretary"" + 0.002*""national"" + 0.002*""bill"" + 0.002*""political"" + 0.002*""senate"" + 0.002*""committee"" + 0.001*""vote""",0.735023
1,"0.004*""british"" + 0.004*""german"" + 0.004*""troops"" + 0.003*""officers"" + 0.002*""colonel"" + 0.002*""military"" + 0.002*""soldiers"" + 0.002*""officer"" + 0.002*""guns"" + 0.001*""sir""",0.732356
34,"0.006*""km"" + 0.005*""national"" + 0.004*""billion"" + 0.004*""million"" + 0.004*""na"" + 0.003*""military"" + 0.003*""population"" + 0.003*""president"" + 0.003*""total"" + 0.003*""economic""",0.715932
7,"0.009*""la"" + 0.004*""et"" + 0.004*""le"" + 0.003*""des"" + 0.002*""les"" + 0.002*""du"" + 0.002*""que"" + 0.002*""en"" + 0.001*""qui"" + 0.001*""e""",0.704893
0,"0.002*""science"" + 0.002*""theory"" + 0.001*""mental"" + 0.001*""conditions"" + 0.001*""religious"" + 0.001*""development"" + 0.001*""principle"" + 0.001*""scientific"" + 0.001*""relation"" + 0.001*""process""",0.614956
5,"0.008*""sir"" + 0.002*""duke"" + 0.002*""thou"" + 0.002*""thy"" + 0.002*""earl"" + 0.002*""ye"" + 0.002*""william"" + 0.002*""hath"" + 0.002*""james"" + 0.002*""thence""",0.607086


In [ ]:
#start coherence func running, mprof plot, save img - after, look at pyLDAvis, run params
#collect in a spreadsheet

In [ ]:
#for demo - look at list of titles associated with each (certain) topic(s)?

In [72]:
#set up filepaths for this model for this task
m = 'gpu_mod_run_4'
fps_test = DirFileMgr(m)
fps_test.create_all_modeling_fps(m)

#load the trained LDA model
lda = LdaModel.load(fps_test.model_fp)

2018-03-29 11:31:05,510 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model
2018-03-29 11:31:05,552 : INFO : loading expElogbeta from ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model.expElogbeta.npy with mmap=None
2018-03-29 11:31:05,605 : INFO : setting ignored attribute id2word to None
2018-03-29 11:31:05,607 : INFO : setting ignored attribute state to None
2018-03-29 11:31:05,608 : INFO : setting ignored attribute dispatcher to None
2018-03-29 11:31:05,610 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model
2018-03-29 11:31:05,612 : INFO : dtype was not set in saved LdaMulticore file ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model, assuming np.float64
2018-03-29 11:31:05,614 : INFO : loading LdaState object from ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model.state


corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_mod_run_params.txt


2018-03-29 11:31:05,947 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model.state
2018-03-29 11:31:05,955 : INFO : dtype was not set in saved LdaState file ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model.state, assuming np.float64


In [74]:
lda.alpha

array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
       0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])

In [75]:
lda.batch

False

In [80]:
lda.bound()

TypeError: bound() missing 1 required positional argument: 'corpus'

In [77]:
lda.chunksize

2000

In [78]:
lda.decay

0.5

In [79]:
lda.diff

<bound method LdaModel.diff of <gensim.models.ldamulticore.LdaMulticore object at 0x146725be0>>

In [81]:
lda.eta

array([0.05, 0.05, 0.05, ..., 0.05, 0.05, 0.05])

In [82]:
lda.eval_every

10

In [83]:
lda.gamma_threshold

0.001

In [84]:
lda.iterations

50

In [85]:
lda.minimum_phi_value

0.01

In [86]:
lda.minimum_probability

0.01

In [87]:
lda.num_terms

262826

In [88]:
lda.num_topics

20

In [89]:
lda.num_updates

28492

In [90]:
lda.numworkers

1

In [91]:
lda.offset

1.0

In [92]:
lda.passes

5

In [95]:
lda.random_state

In [98]:
lda.state

In [100]:
lda.update_every

1

In [99]:
lda.workers

8

In [ ]:
lda.inference

In [16]:
cm1_umass = CoherenceModel(model=lda1, \
                    corpus=serial_corp, \
                    dictionary=d, \
                    coherence='u_mass')

cm1_umass.save(fps1.coherence_model_fp + 'umass')

cm1_umass.get_coherence()

cm1_umass.get_coherence_per_topic()

In [10]:
cm1_uci = CoherenceModel(model=lda1,
                    texts=corp_lst_stream, \
                    coherence='c_uci')

cm1_uci_coherence = cm1_uci.get_coherence())
cm1_uci_topics_coherence = cm1_uci.get_coherence_per_topic())

cm1_uci.save(fps1.coherence_model_fp + 'uci')